# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql.functions import isnan, when, count, col, desc

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
df = pd.read_csv("immigration_data_sample.csv")
df2 = pd.read_csv("us-cities-demographics.csv", sep=';')

In [3]:
df.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [4]:
df2.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


Judging by the analysis below (searching for duplicates, etc.) it appears the "Foriegn-born" column only makes sense when looking at City AND Foreign-born together. When the two tables were joined, duplicate values were made; these should be resolved when making fact and dimension tables.

In [5]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [6]:
#write to parquet
# df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

In [7]:
# Count rows in Immigration table to ensure 1Million+
df_spark.count()

3096313

In [9]:
df_spark.head()

Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')

In [10]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

Convert us-cities-demographics csv into spark dataframe

In [11]:
df2_spark = spark.createDataFrame(df2)
df2_spark.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: double (nullable = true)
 |-- Female Population: double (nullable = true)
 |-- Total Population: long (nullable = true)
 |-- Number of Veterans: double (nullable = true)
 |-- Foreign-born: double (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: long (nullable = true)



#### Cleaning Steps
Document steps necessary to clean the data

In [12]:
# Select relevant columns from Dem data
dem_data = df2_spark.select(
    ["City", "State", "Median Age", "Male Population",
     "Female Population", "Foreign-born", "State Code", "Race"]
)

In [13]:
# Create temp views
df_spark.createOrReplaceTempView("imm_data")
dem_data.createOrReplaceTempView("dem_data")

In [14]:
# Select relevant columns from Immigration data
imm_data = spark.sql("""
    SELECT i94port AS `Entry Port`,
           i94mode AS `Travel Mode`,
           i94addr AS `Dest State`,
           CAST(biryear AS INT) AS `Birth Year`,
           gender AS Gender,
           visatype AS `Visa Type`
    FROM imm_data
""")

imm_data.head(5)

[Row(Entry Port='LOS', Travel Mode=1.0, Dest State='CA', Birth Year=1976, Gender='F', Visa Type='B1'),
 Row(Entry Port='LOS', Travel Mode=1.0, Dest State='NV', Birth Year=1984, Gender='F', Visa Type='B1'),
 Row(Entry Port='LOS', Travel Mode=1.0, Dest State='WA', Birth Year=1987, Gender='M', Visa Type='B1'),
 Row(Entry Port='LOS', Travel Mode=1.0, Dest State='WA', Birth Year=1987, Gender='F', Visa Type='B1'),
 Row(Entry Port='LOS', Travel Mode=1.0, Dest State='WA', Birth Year=1988, Gender='M', Visa Type='B1')]

### Imm_data Schema

In [15]:
# Print new schema
imm_data.printSchema()

root
 |-- Entry Port: string (nullable = true)
 |-- Travel Mode: double (nullable = true)
 |-- Dest State: string (nullable = true)
 |-- Birth Year: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Visa Type: string (nullable = true)



### Dem_data Schema

In [16]:
# Print new schema
dem_data.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: double (nullable = true)
 |-- Female Population: double (nullable = true)
 |-- Foreign-born: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)



In [17]:
# Replace TempView for imm_data
imm_data.createOrReplaceTempView("imm_data")

In [18]:
# Use the `State Code` format (back ticks ``) to select columns
# that contain spaces in their name
spark.sql("""
    SELECT `State Code`
    FROM dem_data
""").show()

+----------+
|State Code|
+----------+
|        MD|
|        MA|
|        AL|
|        CA|
|        NJ|
|        IL|
|        AZ|
|        CA|
|        MO|
|        NC|
|        CA|
|        CA|
|        PA|
|        KS|
|        KS|
|        FL|
|        PA|
|        TX|
|        CA|
|        CA|
+----------+
only showing top 20 rows



In [19]:
# Sanity check
spark.sql("""
    SELECT `Dest State`
    FROM imm_data
    LIMIT 5
""").show()

+----------+
|Dest State|
+----------+
|        CA|
|        NV|
|        WA|
|        WA|
|        WA|
+----------+



In [20]:
# Get gender info and via type from imm_data
spark.sql("""
    SELECT `Entry Port`, Gender, `Visa Type`
    FROM imm_data
    LIMIT 5
""").show()

+----------+------+---------+
|Entry Port|Gender|Visa Type|
+----------+------+---------+
|       LOS|     F|       B1|
|       LOS|     F|       B1|
|       LOS|     M|       B1|
|       LOS|     F|       B1|
|       LOS|     M|       B1|
+----------+------+---------+



In [21]:
# SANITY CHECK imm_data
spark.sql("""
    SELECT DISTINCT `Travel Mode`, `Dest State`, `Birth Year`, Gender, `Visa Type`
    FROM imm_data
""").show()

+-----------+----------+----------+------+---------+
|Travel Mode|Dest State|Birth Year|Gender|Visa Type|
+-----------+----------+----------+------+---------+
|        1.0|        TR|      1959|  null|       B2|
|        1.0|        FL|      1975|     M|       B2|
|        1.0|        TX|      1969|     M|       B2|
|        1.0|        HI|      1936|     M|       WT|
|        1.0|      null|      1988|     F|       WT|
|        1.0|        IL|      1982|     M|       WB|
|        1.0|        HI|      1934|     F|       WT|
|        1.0|        HI|      2004|     F|       WT|
|        1.0|        CA|      1974|     M|       WT|
|        1.0|        CA|      1961|     M|       B1|
|        1.0|        TX|      1971|     M|       B2|
|        1.0|        CA|      1986|     F|       B2|
|        1.0|        NC|      1961|     F|       B2|
|        1.0|        AK|      1951|     F|       B2|
|        1.0|        MA|      1937|     F|       B2|
|        1.0|        MD|      1969|     F|    

In [22]:
# SANITY CHECK dem_data
spark.sql("""
    SELECT DISTINCT City, State, `State Code`, `Foreign-born`, Race
    FROM dem_data
""").show()

+-------------------+--------------+----------+------------+--------------------+
|               City|         State|State Code|Foreign-born|                Race|
+-------------------+--------------+----------+------------+--------------------+
|         Wilmington|North Carolina|        NC|      7401.0|               Asian|
|          Las Vegas|        Nevada|        NV|    127609.0|American Indian a...|
|          Riverside|    California|        CA|     69584.0|               Asian|
|   Town 'n' Country|       Florida|        FL|     22135.0|               Asian|
|             Perris|    California|        CA|     23277.0|               White|
|            Houston|         Texas|        TX|    696210.0|Black or African-...|
|             Tucson|       Arizona|        AZ|     82220.0|  Hispanic or Latino|
|        Centreville|      Virginia|        VA|     27797.0|               White|
|          Elk Grove|    California|        CA|     38693.0|               Asian|
|          San R

In [23]:
# Check for nulls in imm_data
imm_data.printSchema()

root
 |-- Entry Port: string (nullable = true)
 |-- Travel Mode: double (nullable = true)
 |-- Dest State: string (nullable = true)
 |-- Birth Year: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Visa Type: string (nullable = true)



In [24]:
imm_data.filter(imm_data["Birth Year"]=="NaN").show()

+----------+-----------+----------+----------+------+---------+
|Entry Port|Travel Mode|Dest State|Birth Year|Gender|Visa Type|
+----------+-----------+----------+----------+------+---------+
+----------+-----------+----------+----------+------+---------+



In [25]:
dem_data.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: double (nullable = true)
 |-- Female Population: double (nullable = true)
 |-- Foreign-born: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)



In [26]:
# Check for nulls in dem_data
dem_data.filter(dem_data["Foreign-born"]=="NaN").show()

+--------+-----------+----------+---------------+-----------------+------------+----------+--------------------+
|    City|      State|Median Age|Male Population|Female Population|Foreign-born|State Code|                Race|
+--------+-----------+----------+---------------+-----------------+------------+----------+--------------------+
|San Juan|Puerto Rico|      41.4|       155408.0|         186829.0|         NaN|        PR|  Hispanic or Latino|
|  Caguas|Puerto Rico|      40.4|        34743.0|          42265.0|         NaN|        PR|  Hispanic or Latino|
|Carolina|Puerto Rico|      42.0|        64758.0|          77308.0|         NaN|        PR|American Indian a...|
|Carolina|Puerto Rico|      42.0|        64758.0|          77308.0|         NaN|        PR|  Hispanic or Latino|
|San Juan|Puerto Rico|      41.4|       155408.0|         186829.0|         NaN|        PR|American Indian a...|
|Mayagüez|Puerto Rico|      38.1|        30799.0|          35782.0|         NaN|        PR|     

In [27]:
# Filter out NaN in dem_data
dem_data = dem_data.filter(dem_data["Foreign-born"]!="NaN")

In [28]:
# Sanity check
dem_data.filter(dem_data["Foreign-born"]=="NaN").show()

+----+-----+----------+---------------+-----------------+------------+----------+----+
|City|State|Median Age|Male Population|Female Population|Foreign-born|State Code|Race|
+----+-----+----------+---------------+-----------------+------------+----------+----+
+----+-----+----------+---------------+-----------------+------------+----------+----+



In [29]:
# Recheck
dem_data.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: double (nullable = true)
 |-- Female Population: double (nullable = true)
 |-- Foreign-born: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)



In [30]:
dem_data.filter(dem_data["Male Population"]=="NaN").show()

+------------+-------+----------+---------------+-----------------+------------+----------+--------------------+
|        City|  State|Median Age|Male Population|Female Population|Foreign-born|State Code|                Race|
+------------+-------+----------+---------------+-----------------+------------+----------+--------------------+
|The Villages|Florida|      70.5|            NaN|              NaN|      4034.0|        FL|  Hispanic or Latino|
|The Villages|Florida|      70.5|            NaN|              NaN|      4034.0|        FL|Black or African-...|
|The Villages|Florida|      70.5|            NaN|              NaN|      4034.0|        FL|               White|
+------------+-------+----------+---------------+-----------------+------------+----------+--------------------+



In [31]:
# Remove NaN values
dem_data = dem_data.filter(dem_data["Male Population"]!="NaN")

In [32]:
# Confirm removal
dem_data.filter(dem_data["Male Population"]=="NaN").show()

+----+-----+----------+---------------+-----------------+------------+----------+----+
|City|State|Median Age|Male Population|Female Population|Foreign-born|State Code|Race|
+----+-----+----------+---------------+-----------------+------------+----------+----+
+----+-----+----------+---------------+-----------------+------------+----------+----+



In [33]:
# Check for NaN from Female Population
dem_data.filter(dem_data["Female Population"]=="NaN").show()

+----+-----+----------+---------------+-----------------+------------+----------+----+
|City|State|Median Age|Male Population|Female Population|Foreign-born|State Code|Race|
+----+-----+----------+---------------+-----------------+------------+----------+----+
+----+-----+----------+---------------+-----------------+------------+----------+----+



In [34]:
# Replace views
imm_data.createOrReplaceTempView("imm_data")
dem_data.createOrReplaceTempView("dem_data")

In [35]:
imm_dem_table = spark.sql("""
    SELECT d.City, d.State, d.`State Code`, i.`Travel Mode`,
           i.Gender, d.`Foreign-born`, d.Race, i.`Visa Type`
    FROM imm_data i
    JOIN dem_data d ON d.`State Code` = i.`Dest State`
""")

In [36]:
imm_dem_table.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Travel Mode: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Foreign-born: double (nullable = true)
 |-- Race: string (nullable = true)
 |-- Visa Type: string (nullable = true)



In [37]:
# Sanity check: See what we're working with
imm_dem_table.show()

+-------------+--------+----------+-----------+------+------------+------------------+---------+
|         City|   State|State Code|Travel Mode|Gender|Foreign-born|              Race|Visa Type|
+-------------+--------+----------+-----------+------+------------+------------------+---------+
|Silver Spring|Maryland|        MD|        1.0|     M|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|     F|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|     M|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|     M|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|     M|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|     F|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|     F|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|      

In [38]:
# Filter out null values
imm_dem_table.filter("Gender is null").show()

+-------------+--------+----------+-----------+------+------------+------------------+---------+
|         City|   State|State Code|Travel Mode|Gender|Foreign-born|              Race|Visa Type|
+-------------+--------+----------+-----------+------+------------+------------------+---------+
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WB|
|Silver Spring|Maryland|        MD|        1.0|  null|     30908.0|Hispanic or Latino|       WT|
|Silver Spring|Maryland|      

In [39]:
# Drop these null values
imm_dem_table = imm_dem_table.filter(imm_dem_table.Gender.isNotNull())

In [40]:
# Sanity check
imm_dem_table.filter("Gender is null").show()

+----+-----+----------+-----------+------+------------+----+---------+
|City|State|State Code|Travel Mode|Gender|Foreign-born|Race|Visa Type|
+----+-----+----------+-----------+------+------------+----+---------+
+----+-----+----------+-----------+------+------------+----+---------+



In [41]:
# Find more null values
imm_dem_table.filter(imm_dem_table["Foreign-born"]=="NaN").show()

+----+-----+----------+-----------+------+------------+----+---------+
|City|State|State Code|Travel Mode|Gender|Foreign-born|Race|Visa Type|
+----+-----+----------+-----------+------+------------+----+---------+
+----+-----+----------+-----------+------+------------+----+---------+



In [42]:
# Create temp view
imm_dem_table.createOrReplaceTempView("imm_dem_table")

In [43]:
# Test query: Return Gender and Foreign born, order by State
spark.sql("""
    SELECT DISTINCT Gender, `Foreign-born`, City, State, Race, `Visa Type`
    FROM imm_dem_table
    ORDER BY State
""").show()

+------+------------+----------+-------+--------------------+---------+
|Gender|Foreign-born|      City|  State|                Race|Visa Type|
+------+------------+----------+-------+--------------------+---------+
|     F|      4706.0|Tuscaloosa|Alabama|Black or African-...|       WB|
|     F|     12691.0|Huntsville|Alabama|Black or African-...|        I|
|     X|      9337.0|Montgomery|Alabama|               Asian|       F1|
|     F|      8258.0|Birmingham|Alabama|Black or African-...|       WT|
|     M|      8258.0|Birmingham|Alabama|American Indian a...|        I|
|     M|      9337.0|Montgomery|Alabama|American Indian a...|       WB|
|     M|      9337.0|Montgomery|Alabama|American Indian a...|        I|
|     F|      4706.0|Tuscaloosa|Alabama|               White|       F1|
|     M|      8258.0|Birmingham|Alabama|               White|       F1|
|     M|      8229.0|    Hoover|Alabama|Black or African-...|       B1|
|     F|      7234.0|    Mobile|Alabama|American Indian a...|   

In [44]:
# View Foreign-born, by city, and with visa type, and race
spark.sql("""
    SELECT City, State,`Foreign-born`, Race, `Visa Type`
    FROM imm_dem_table
    GROUP BY 1, 2, 3, 4, 5
    LIMIT 10
""").show()

+-------------+----------+------------+--------------------+---------+
|         City|     State|Foreign-born|                Race|Visa Type|
+-------------+----------+------------+--------------------+---------+
|     O'Fallon|  Missouri|      3269.0|  Hispanic or Latino|       B1|
|  Santa Clara|California|     52281.0|               White|       F1|
|    Frederick|  Maryland|     14211.0|               White|       E1|
|     Lakewood|  Colorado|     14169.0|  Hispanic or Latino|       F2|
|        Omaha|  Nebraska|     48263.0|Black or African-...|       E2|
|      Killeen|     Texas|     15769.0|American Indian a...|       F1|
|         Waco|     Texas|     14235.0|Black or African-...|       WT|
|      Garland|     Texas|     62975.0|               Asian|       F1|
|  Bloomington|  Illinois|      7287.0|               Asian|       E2|
|Wichita Falls|     Texas|      9855.0|American Indian a...|      SBP|
+-------------+----------+------------+--------------------+---------+



In [45]:
# Check new schema
imm_dem_table.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Travel Mode: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Foreign-born: double (nullable = true)
 |-- Race: string (nullable = true)
 |-- Visa Type: string (nullable = true)



In [46]:
# Get an idea of the data we have
spark.sql("""
    SELECT *
    FROM imm_dem_table
    ORDER BY City
    LIMIT 5
""").show()

+-------+-----+----------+-----------+------+------------+--------------------+---------+
|   City|State|State Code|Travel Mode|Gender|Foreign-born|                Race|Visa Type|
+-------+-----+----------+-----------+------+------------+--------------------+---------+
|Abilene|Texas|        TX|        1.0|     M|      8129.0|American Indian a...|       WB|
|Abilene|Texas|        TX|        1.0|     F|      8129.0|American Indian a...|       WT|
|Abilene|Texas|        TX|        1.0|     M|      8129.0|American Indian a...|       WT|
|Abilene|Texas|        TX|        1.0|     M|      8129.0|American Indian a...|       WT|
|Abilene|Texas|        TX|        1.0|     F|      8129.0|American Indian a...|       WT|
+-------+-----+----------+-----------+------+------------+--------------------+---------+



In [47]:
# Find duplicate data
imm_dem_table.groupBy("City", "Race").count().sort(desc("count")).show()

+----------------+--------------------+------+
|            City|                Race| count|
+----------------+--------------------+------+
|    Jacksonville|  Hispanic or Latino|556235|
|    Jacksonville|               Asian|556235|
|    Jacksonville|American Indian a...|556235|
|    Jacksonville|Black or African-...|556235|
|    Jacksonville|               White|556235|
| West Palm Beach|               White|535516|
|        Palm Bay|Black or African-...|535516|
| Fort Lauderdale|               Asian|535516|
|           Davie|  Hispanic or Latino|535516|
|Saint Petersburg|  Hispanic or Latino|535516|
|       Riverview|               White|535516|
|     Spring Hill|American Indian a...|535516|
|       Homestead|Black or African-...|535516|
|Saint Petersburg|Black or African-...|535516|
|      Palm Coast|Black or African-...|535516|
|       Melbourne|American Indian a...|535516|
|       Poinciana|               Asian|535516|
|      Cape Coral|American Indian a...|535516|
|   Coral Spr

In [48]:
imm_dem_table = imm_dem_table.dropDuplicates()

In [49]:
# Recheck (should be 186 or less for each row)
imm_dem_table.groupBy("City", "Race").count().sort(desc("count")).show()

+-----------+--------------------+-----+
|       City|                Race|count|
+-----------+--------------------+-----+
|   Pasadena|  Hispanic or Latino|  186|
|   Pasadena|               White|  186|
|   Pasadena|Black or African-...|  186|
|   Pasadena|               Asian|  186|
|   Pasadena|American Indian a...|  186|
| Union City|               Asian|  161|
| Union City|Black or African-...|  161|
| Union City|               White|  161|
| Union City|American Indian a...|  161|
| Union City|  Hispanic or Latino|  161|
|   Glendale|               Asian|  156|
|   Glendale|Black or African-...|  156|
|   Glendale|               White|  156|
|   Glendale|  Hispanic or Latino|  156|
|   Glendale|American Indian a...|  156|
|Springfield|               White|  151|
|Springfield|               Asian|  151|
|Springfield|American Indian a...|  151|
|Springfield|Black or African-...|  151|
|Springfield|  Hispanic or Latino|  151|
+-----------+--------------------+-----+
only showing top

In [50]:
# Ultimate check: Count new rows
spark.sql("""
    SELECT COUNT(*)
    FROM imm_dem_table
""").show()

+---------+
| count(1)|
+---------+
|496121831|
+---------+



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

In [52]:
# Map out City table
spark.sql("""
    SELECT DISTINCT City, State, `State Code`, Race
    FROM dem_data
    ORDER BY State
""").show()

+----------+-------+----------+--------------------+
|      City|  State|State Code|                Race|
+----------+-------+----------+--------------------+
|    Hoover|Alabama|        AL|Black or African-...|
|Tuscaloosa|Alabama|        AL|American Indian a...|
|Tuscaloosa|Alabama|        AL|  Hispanic or Latino|
|    Dothan|Alabama|        AL|American Indian a...|
|    Mobile|Alabama|        AL|Black or African-...|
|Birmingham|Alabama|        AL|Black or African-...|
|Birmingham|Alabama|        AL|               White|
|    Hoover|Alabama|        AL|  Hispanic or Latino|
|Huntsville|Alabama|        AL|Black or African-...|
|Montgomery|Alabama|        AL|Black or African-...|
|    Mobile|Alabama|        AL|               Asian|
|Huntsville|Alabama|        AL|American Indian a...|
|Montgomery|Alabama|        AL|               Asian|
|    Mobile|Alabama|        AL|               White|
|Tuscaloosa|Alabama|        AL|               White|
|    Dothan|Alabama|        AL|               

In [53]:
# Map Entry Port table
spark.sql("""
    SELECT DISTINCT `Entry Port`, CAST(`Travel Mode` AS INT), `Dest State`, `Visa Type`
    FROM imm_data
    WHERE `Dest State` != "null"
    ORDER BY 1
""").show()

+----------+-----------+----------+---------+
|Entry Port|Travel Mode|Dest State|Visa Type|
+----------+-----------+----------+---------+
|       5KE|          2|        AK|       B2|
|       5T6|          1|        NM|       B2|
|       5T6|          1|        TX|       B2|
|       ABG|          3|        VT|       B2|
|       ABG|          3|        VT|       WT|
|       ABG|          1|        GU|      GMT|
|       ABG|          3|        MA|       WT|
|       ABQ|          1|        NM|       M1|
|       ABQ|          1|        NM|       F1|
|       ABS|          3|        MA|       WB|
|       ABS|          3|        CT|       B2|
|       ABS|          3|        VT|       WT|
|       ADS|          1|        TX|       B2|
|       ADS|          1|        TX|       B1|
|       ADT|          3|        TX|       B2|
|       ADW|          1|        MD|        I|
|       AGA|          1|        GQ|       WB|
|       AGA|          1|        AN|       B2|
|       AGA|          1|        FL

In [57]:
# Map State table
spark.sql("""
        SELECT DISTINCT State, CAST(SUM(`Male Population`) AS INT) AS `Male Pop Total`,
               CAST(SUM(`Female Population`) AS INT) AS `Fem Pop Total`,
               CAST(SUM(`Foreign-born`) AS INT) AS `Foreign-born Tot`
        FROM dem_data
        GROUP BY 1
        ORDER BY State, `Foreign-born tot` DESC
""").show()

+--------------------+--------------+-------------+----------------+
|               State|Male Pop Total|Fem Pop Total|Foreign-born Tot|
+--------------------+--------------+-------------+----------------+
|             Alabama|       2448200|      2715106|          252541|
|              Alaska|        764725|       728750|          166290|
|             Arizona|      11137275|     11360435|         3411565|
|            Arkansas|       1400724|      1482165|          307753|
|          California|      61055672|     62388681|        37059662|
|            Colorado|       7273095|      7405250|         1688155|
|         Connecticut|       2123435|      2231661|         1114250|
|            Delaware|        163400|       196385|           16680|
|District of Columbia|       1598525|      1762615|          475585|
|             Florida|      15461937|     16626425|         7833464|
|             Georgia|       4101605|      4453555|          738925|
|              Hawaii|        8840

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.